# Task 3 - Build Summary Tables

#### Imports

In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account

import pandas as pd
from pandas.io import gbq

#### Connect to GBQ

In [2]:
service_path = ' '
service_file = ' ' # change this to your authentication information  
gbq_proj_id = ' ' # change this to your project. 
#gbq_dataset_id = ' ' # and change this to your data set ID
gbq_dataset_id = ' ' # and change this to your data set ID

# Creates private key 
private_key =service_path + service_file


#### Credentials

In [3]:
# Get your credentials
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And create a client to talk to GBQ
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

###  Run queries and create text files to be used in database creation

#### Question 1 - Sales by date by hour

In [4]:
# Runs query

q1 = """
SELECT (EXTRACT(date FROM datetime)) AS Date,
(EXTRACT(hour FROM datetime)) AS Hour,
SUM(total) AS Sales,
COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items

FROM `wedgehp.other.records_from_sample`

WHERE card_no != 3
AND department != 0
AND department != 15
AND trans_status != 'M'
AND trans_status != 'C'
AND trans_status != 'J'
AND (trans_status = ''
OR trans_status = ' '
OR trans_status =  'V'
OR trans_status = 'R')

GROUP BY Date, Hour
ORDER BY Date ASC, Hour DESC
"""

t3q1 = gbq.read_gbq(q1, project_id ="wedgehp")



In [11]:
# Checks to make sure it worked
t3q1.head(5)

,Date,Hour,Sales,Transactions,Items
0,2010-01-01,17,28.12,1,8
1,2010-01-01,15,82.12,1,25
2,2010-01-01,14,111.55,4,42
3,2010-01-01,13,90.28,3,27
4,2010-01-01,12,60.89,2,13


In [19]:
# Saves file to be used in db creation
t3q1.to_csv(r'sales_by_date_by_hour.txt',header=None, index=None, sep=',', mode='a')

#### Question 2 - Sales by owner by year by month

In [7]:
# Runs query

q2 = """
SELECT card_no as owner,
(EXTRACT(year FROM datetime)) AS Year,
(EXTRACT(month FROM datetime)) AS Month,
SUM(total) AS Sales,
COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items

FROM `wedgehp.other.records_from_sample` 

WHERE card_no != 3
AND department != 0
AND department != 15
AND trans_status != 'M'
AND trans_status != 'C'
AND trans_status != 'J'
AND (trans_status = ''
OR trans_status = ' '
OR trans_status =  'V'
OR trans_status = 'R')

GROUP BY Year, Month, owner
ORDER BY Year ASC , Month ASC

"""

t3q2 = gbq.read_gbq(q2, project_id ="wedgehp")


In [8]:
# Checks to make sure it worked
t3q2.head(5)

,owner,Year,Month,Sales,Transactions,Items
0,16143.0,2010,1,928.04,16,240
1,25395.0,2010,1,1624.29,59,460
2,23676.0,2010,1,744.62,7,190
3,18003.0,2010,1,550.48,7,85
4,20914.0,2010,1,131.10,3,37


In [20]:
# Saves file to be used in db creation
t3q2.to_csv(r'sales_by_owner_by_year_by_month.txt', header=None,index=None, sep=',', mode='a')

#### Question 3 - Sales by product description by year by month

In [13]:
# Runs query
q3 = """

SELECT upc, description, A.department, B.department_name,
(EXTRACT(year FROM datetime)) AS Year,
(EXTRACT(month FROM datetime)) AS Month,
SUM(total) AS Sales,
COUNT(DISTINCT(Date(datetime) || register_no || emp_no || trans_no)) AS Transactions,
SUM(CASE WHEN(trans_status = 'V' OR trans_status = 'R') THEN -1 ELSE 1 END) as Items

FROM `wedgehp.other.records_from_sample` A
JOIN `wedgehp.other.departments` B ON A.department = B.department

WHERE card_no != 3
AND a.department != 0
AND a.department != 15
AND trans_status != 'M'
AND trans_status != 'C'
AND trans_status != 'J'
AND (trans_status = ''
OR trans_status = ' '
OR trans_status =  'V'
OR trans_status = 'R')

GROUP BY Year, Month, description, A.department, B.department_name, upc
ORDER BY Year ASC, Month ASC

"""

t3q3 = gbq.read_gbq(q3, project_id ="wedgehp")


In [21]:
# Checks to make sure it worked
t3q3.head(5)

,upc,description,department,department_name,Year,Month,Sales,Transactions,Items
0,0.21DP4,REF GROCERY,4.000000000,REF GROCERY,2010,1,0.00,1,0
1,0.89DP9,GEN MERCH,9.000000000,GEN MERCH,2010,1,1.78,2,2
2,1.98DP11,PERSONAL CARE,11.000000000,PERSONAL CARE,2010,1,1.98,1,1
3,38.02DP3,BULK,3.000000000,BULK,2010,1,76.04,2,0
4,4.07DP13,MEAT,13.000000000,MEAT,2010,1,4.07,1,1


In [18]:
# Saves dile for db creation
t3q3.to_csv(r'sales_by_product_description_by_year_by_month.txt', header=None, index=None, sep=',', mode='a')